# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

The data represents 17 marketing campaigns conducted between May 2008 and November 2010.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [30]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [5]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [6]:
missing_values = df.isnull().sum()
print(missing_values)


age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64


In [7]:
data_types = df.dtypes
print(data_types)


age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Business Objective

The business objective of the task is to develop a predictive model that can accurately determine whether a bank client will subscribe to a term deposit based on various features. The model aims to analyze client data, information related to the last contact of the current campaign, and other relevant attributes to predict the likelihood of a client subscribing to a term deposit.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

* Encode Categorical Variables: Since columns 2 to 7 are categorical, we can use one-hot encoding to transform these data.



In [9]:
df_encoded = pd.get_dummies(df[['job', 'marital', 'education', 'default', 'housing', 'loan']])
df_encoded

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
0,False,False,False,True,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,True,False,True,False,False,True,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,True,True,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
4,False,False,False,False,False,False,False,True,False,False,...,False,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,False,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,True,True,False,False
41184,False,True,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
41185,False,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,True,True,False,False
41186,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,True,False,False,True,False,False


* Scale Numeric Variables: Column 1, ‘age’, is numeric. It'll be beneficial to scale numeric features to ensure that they have a similar impact on the model.

In [10]:
scaler = StandardScaler()
df['age'] = scaler.fit_transform(df[['age']])


* Concatenate Encoded Features with Numeric Features

In [11]:
numeric_features = df[['age']]

In [12]:
df_final = pd.concat([numeric_features, df_encoded], axis=1)

In [13]:
df_final

,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
0,1.533034,False,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,1.628993,False,False,False,False,False,False,False,True,False,...,False,False,True,False,True,False,False,True,False,False
2,-0.290186,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,True,True,False,False
3,-0.002309,True,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
4,1.533034,False,False,False,False,False,False,False,True,False,...,False,True,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,3.164336,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
41184,0.573445,False,True,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
41185,1.533034,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
41186,0.381527,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [14]:
labelEncoder = LabelEncoder()
df['y'] = labelEncoder.fit_transform(df['y'])

df_final['target'] = df['y']

In [15]:
df_final

,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,target
0,1.533034,False,False,False,True,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,0
1,1.628993,False,False,False,False,False,False,False,True,False,...,False,True,False,True,False,False,True,False,False,0
2,-0.290186,False,False,False,False,False,False,False,True,False,...,True,False,False,False,False,True,True,False,False,0
3,-0.002309,True,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,0
4,1.533034,False,False,False,False,False,False,False,True,False,...,True,False,False,True,False,False,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,3.164336,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,True,True,False,False,1
41184,0.573445,False,True,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,0
41185,1.533034,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,True,True,False,False,0
41186,0.381527,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,1


In [16]:
X = df_final.drop('target', axis=1)
y = df_final['target']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [18]:
print(y.unique())
print(y.value_counts())


[0 1]
target
0    36548
1     4640
Name: count, dtype: int64


In [19]:
baseline_accuracy = y.value_counts(normalize=True)[0]
print(f"Baseline Accuracy: {baseline_accuracy:.2%}")

Baseline Accuracy: 88.73%


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [20]:
model = LogisticRegression(random_state=42)

start_time = time.time()

model.fit(X_train, y_train)

train_time = time.time() - start_time

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)



/Users/jay/miniconda3/envs/myenv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Problem 9: Score the Model

What is the accuracy of your model?

In [22]:
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Time: {train_time:.4f} seconds")
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy: {test_accuracy:.2%}")

Training Time: 0.5928 seconds
Training Accuracy: 88.76%
Test Accuracy: 88.65%


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

### Decision Tree


In [23]:
from sklearn.tree import DecisionTreeClassifier


model = DecisionTreeClassifier(random_state=42)

start_time = time.time()

model.fit(X_train, y_train)

train_time = time.time() - start_time

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Time: {train_time:.4f} seconds")
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy: {test_accuracy:.2%}")


Training Time: 0.0936 seconds
Training Accuracy: 91.66%
Test Accuracy: 86.25%


### SVC


In [24]:
from sklearn.svm import SVC


model = SVC(random_state=42)

start_time = time.time()

model.fit(X_train, y_train)

train_time = time.time() - start_time

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Time: {train_time:.4f} seconds")
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy: {test_accuracy:.2%}")


Training Time: 34.8717 seconds
Training Accuracy: 88.82%
Test Accuracy: 88.67%


### KNN

In [25]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

X = df_final.iloc[:, 0:7].values
y = y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn_model = KNeighborsClassifier(n_neighbors=5)

start_time = time.time()

knn_model.fit(X_train, y_train)

train_time = time.time() - start_time

y_train_pred = knn_model.predict(X_train)
y_test_pred = knn_model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Time: {train_time:.4f} seconds")
print(f"Training Accuracy: {train_accuracy:.2%}")
print(f"Test Accuracy: {test_accuracy:.2%}")


Training Time: 0.0224 seconds
Training Accuracy: 87.30%
Test Accuracy: 87.34%


### Model Comparison

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|Logistic Regression|0.5928 sec|88.76%|88.65%
|Decision Tree|0.0936 sec|91.66%|86.25%
|SVC|34.8717 sec|88.82%|88.67%
|KNN|0.0224 sec|87.30%|87.34%


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression
logistic_params = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
logistic_model = LogisticRegression(random_state=42)
grid_logistic = GridSearchCV(logistic_model, logistic_params, cv=5)
grid_logistic.fit(X_train, y_train)
logistic_best_model = grid_logistic.best_estimator_
logistic_y_pred = logistic_best_model.predict(X_test)
logistic_accuracy = accuracy_score(y_test, logistic_y_pred)
print("Logistic Regression Best Parameters:", grid_logistic.best_params_)
print("Logistic Regression Test Accuracy:", logistic_accuracy)

# KNN
knn_params = {'n_neighbors': [3, 5, 7, 9], 'weights': ['uniform', 'distance']}
knn_model = KNeighborsClassifier()
grid_knn = GridSearchCV(knn_model, knn_params, cv=5)
grid_knn.fit(X_train, y_train)
knn_best_model = grid_knn.best_estimator_
knn_y_pred = knn_best_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_y_pred)
print("\nK-Nearest Neighbors Best Parameters:", grid_knn.best_params_)
print("K-Nearest Neighbors Test Accuracy:", knn_accuracy)

# Decision Tree
tree_params = {'max_depth': [None, 5, 10, 15], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
tree_model = DecisionTreeClassifier(random_state=42)
grid_tree = GridSearchCV(tree_model, tree_params, cv=5)
grid_tree.fit(X_train, y_train)
tree_best_model = grid_tree.best_estimator_
tree_y_pred = tree_best_model.predict(X_test)
tree_accuracy = accuracy_score(y_test, tree_y_pred)
print("\nDecision Tree Best Parameters:", grid_tree.best_params_)
print("Decision Tree Test Accuracy:", tree_accuracy)


Logistic Regression Best Parameters: {'C': 0.001, 'penalty': 'l2'}
Logistic Regression Test Accuracy: 0.8865015780529255

K-Nearest Neighbors Best Parameters: {'n_neighbors': 9, 'weights': 'distance'}
K-Nearest Neighbors Test Accuracy: 0.8854090798737557

Decision Tree Best Parameters: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 10}
Decision Tree Test Accuracy: 0.8855304685603301


The GridSearchCV process was time-consuming due to its computational expense. To address this issue, I opted in for RandomizedSearchCV instead. While it may sacrifice a bit of accuracy, the trade-off could be negligible, depending on the specific problem and dataset size, and it generally offers faster performance.

In [ ]:
# svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
# svm_model = SVC(random_state=42)
# grid_svm = GridSearchCV(svm_model, svm_params, cv=5, n_jobs=-1)
# grid_svm.fit(X_train, y_train)
# svm_best_model = grid_svm.best_estimator_
# svm_y_pred = svm_best_model.predict(X_test)
# svm_accuracy = accuracy_score(y_test, svm_y_pred)
# print("\nSVM Best Parameters:", grid_svm.best_params_)
# print("SVM Test Accuracy:", svm_accuracy)


In [27]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_params = {'C': uniform(loc=0.1, scale=2), 'kernel': ['linear', 'rbf']}

svm_model = SVC(random_state=42)
randomized_svm = RandomizedSearchCV(svm_model, svm_params, n_iter=10, cv=5, random_state=42, n_jobs=-1)
randomized_svm.fit(X_train, y_train)

svm_best_model = randomized_svm.best_estimator_
svm_best_params = randomized_svm.best_params_

svm_y_pred = svm_best_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_y_pred)

print("Randomized SVM Best Parameters:", svm_best_params)
print("Randomized SVM Test Accuracy:", svm_accuracy)



Randomized SVM Best Parameters: {'C': 0.849080237694725, 'kernel': 'linear'}
Randomized SVM Test Accuracy: 0.8865015780529255


Upon executing GridSearchCV, certain models exhibited a marginal decrease in performance (less than 1%), while others demonstrated a slight improvement (also less than 1%). The decline in performance could be attributed to potential overfitting, or it might fall within the anticipated range of variation. Alternatively, the GridSearchCV exploration of the hyperparameter space may not have encompassed the optimal settings.

##### Questions